In [ ]:
import cv2
import time
import threading
from flask import Flask, Response, render_template_string
from ultralytics import YOLO
from datetime import datetime

# Load YOLOv8 model
model = YOLO("s1.pt")

# Flask app
app = Flask(__name__)

# Shared frames and locks
frames = {"CAM 1": None, "CAM 2": None}
locks = {"CAM 1": threading.Lock(), "CAM 2": threading.Lock()}

# HTML Template for dual camera view
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>YOLOv8 Snake Detection</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body {
            background: #111;
            color: white;
            font-family: sans-serif;
            margin: 0;
            padding: 0;
        }
        h1 {
            text-align: center;
            padding: 20px;
            font-size: 28px;
        }
        .cam-box {
            background: #222;
            padding: 10px;
            margin: 20px auto;
            max-width: 960px;
            border-radius: 12px;
            box-shadow: 0 0 10px rgba(255,255,255,0.1);
        }
        .cam-box h3 {
            margin: 0 0 10px 0;
            text-align: center;
        }
        .cam-box img {
        display: block;
        border-radius: 10px;
        width: auto;
        height: auto;
        max-width: 100%;
        }
    </style>
</head>
<body>
    <h1>🐍 YOLOv8 Snake Detection Feed</h1>

    <div class="cam-box">
        <h3>CAM 1</h3>
        <img src="{{ url_for('video_feed', cam_name='CAM 1') }}">
    </div>

    <div class="cam-box">
        <h3>CAM 2</h3>
        <img src="{{ url_for('video_feed', cam_name='CAM 2') }}">
    </div>
</body>
</html>
"""

def draw_overlay(frame, cam_name):
    hour = datetime.now().hour
    mode = "Night" if hour >= 22 or hour < 9 else "Day"
    datetime_str = datetime.now().strftime("%m/%d/%Y %H:%M:%S")

    overlay_lines = [f"{cam_name}", datetime_str, f"Mode: {mode}"]

    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    color = (0, 0, 255)
    thickness = 2
    y_offset = 25

    for i, line in enumerate(overlay_lines):
        size = cv2.getTextSize(line, font, scale, thickness)[0]
        x = frame.shape[1] - size[0] - 10
        y = (i + 1) * y_offset
        cv2.putText(frame, line, (x, y), font, scale, color, thickness, cv2.LINE_AA)

    return frame

def detect_and_update(cam_id, cam_name):
    cap = cv2.VideoCapture(cam_id)
    cap.set(3, 640)
    cap.set(4, 480)

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        results = model(frame, verbose=False)[0]

        for box in results.boxes:
            cls = int(box.cls)
            label = model.names[cls]
            conf = float(box.conf)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        with locks[cam_name]:
            frames[cam_name] = frame.copy()

def generate_stream(cam_name):
    while True:
        with locks[cam_name]:
            frame = frames[cam_name]
        if frame is None:
            continue
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n')
        time.sleep(0.5)

@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/video_feed/<cam_name>')
def video_feed(cam_name):
    return Response(generate_stream(cam_name),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    # Start each camera stream in its own thread
    t1 = threading.Thread(target=detect_and_update, args=(0, "CAM 1"), daemon=True)
    t2 = threading.Thread(target=detect_and_update, args=(1, "CAM 2"), daemon=True)
    t1.start()
    t2.start()

    app.run(host='0.0.0.0', port=5000, threaded=True)

In [3]:
import cv2
import os
import time
from datetime import datetime
from threading import Thread, Lock
from flask import Flask, Response, render_template_string
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("s1.pt")  # Change to absolute path if needed

# Settings
fps = 15.0
frame_size = (1260, 720)

app = Flask(__name__)
frames = {"CAM 1": None, "CAM 2": None}
locks = {"CAM 1": Lock(), "CAM 2": Lock()}

html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Snake Stream</title>
    <style>
        body {
            font-family: sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 20px;
            text-align: center;
        }
        h2 {
            margin-bottom: 20px;
        }
        .container {
            display: flex;
            justify-content: center;
            flex-wrap: wrap;
            gap: 20px;
        }
        .cam-box {
            background: white;
            padding: 10px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.2);
        }
        .cam-box h3 {
            margin-bottom: 10px;
        }
        .cam-box img {
            max-width: 100%;
            height: auto;
            width: 960px;
            border-radius: 8px;
        }
    </style>
</head>
<body>
    <h2>Live Camera Feeds</h2>
    <div class="container">
        <div class="cam-box">
            <h3>CAM 1</h3>
            <img src="{{ url_for('video_feed', cam_name='CAM 1') }}">
        </div>
        <div class="cam-box">
            <h3>CAM 2</h3>
            <img src="{{ url_for('video_feed', cam_name='CAM 2') }}">
        </div>
    </div>
</body>
</html>
"""

def adjust_brightness(cap):
    hour = datetime.now().hour
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 0.2 if hour >= 22 or hour < 9 else 1.0)
    return "night" if hour >= 22 or hour < 9 else "day"

def draw_timestamp_overlay(frame, brightness_mode, cam_name):
    now = datetime.now()
    datetime_str = now.strftime("%m/%d/%y %H:%M:%S")
    label = "Day" if brightness_mode == "day" else "Night"
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    color = (0, 0, 255)
    thickness = 2
    lines = [cam_name.upper(), datetime_str, label]
    y_offset = 25
    for i, line in enumerate(lines):
        size = cv2.getTextSize(line, font, scale, thickness)[0]
        x = frame.shape[1] - size[0] - 10
        y = (i + 1) * y_offset
        cv2.putText(frame, line, (x, y), font, scale, color, thickness, cv2.LINE_AA)
    return frame
    
def draw_yolo_predictions(frame):
    results = model(frame, verbose=False)[0]
    for box in results.boxes:
        cls = int(box.cls)
        label = model.names[cls]
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return frame

def monitor_camera(device_path, cam_name):
    cap = cv2.VideoCapture(device_path)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_size[0])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_size[1])
    cap.set(cv2.CAP_PROP_FPS, fps)

    ret, prev_frame = cap.read()
    if not ret:
        print(f"[{cam_name}] ? Failed to access camera.")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.GaussianBlur(prev_gray, (21, 21), 0)

    print(f"[{cam_name}] ?? Watching for motion...")

    while True:
        brightness_mode = adjust_brightness(cap)
        ret, frame = cap.read()
        if not ret:
            cap.release()
            time.sleep(1)
            cap = cv2.VideoCapture(device_path)
            continue

        # Motion detection (can be used for other logic)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        delta = cv2.absdiff(prev_gray, gray)
        thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        prev_gray = gray

        # Add timestamp
        frame = draw_timestamp_overlay(frame, brightness_mode, cam_name)

        # Run YOLO detection and draw boxes
        frame = draw_yolo_predictions(frame)

        # Update Flask stream
        with locks[cam_name]:
            frames[cam_name] = frame.copy()

def generate_stream(cam_name):
    while True:
        with locks[cam_name]:
            frame = frames.get(cam_name)
            if frame is None:
                continue
            _, jpeg = cv2.imencode('.jpg', frame)
        time.sleep(0.03)  # ~30 FPS update
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n')
                   
@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/video_feed/<cam_name>')
def video_feed(cam_name):
    return Response(generate_stream(cam_name), mimetype='multipart/x-mixed-replace; boundary=frame')

# Start threads (Windows: use integers for webcams)
thread1 = Thread(target=monitor_camera, args=(0, "CAM 1"))
thread2 = Thread(target=monitor_camera, args=(1, "CAM 2"))  # Optional second cam
thread1.start()
thread2.start()

# Run Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off
[CAM 2] ? Failed to access camera.


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.140:5000
Press CTRL+C to quit


[CAM 1] ?? Watching for motion...


127.0.0.1 - - [11/Apr/2025 23:20:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2025 23:20:26] "GET /video_feed/CAM%201 HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2025 23:20:26] "GET /favicon.ico HTTP/1.1" 404 -


Exception in thread Thread-21 (monitor_camera):
Traceback (most recent call last):
  File "c:\Users\gkrul\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\gkrul\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gkrul\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gkrul\AppData\Local\Temp\ipykernel_20552\3843434059.py", line 147, in monitor_camera
  File "C:\Users\gkrul\AppData\Local\Temp\ipykernel_20552\3843434059.py", line 98, in draw_yolo_predictions
  File "c:\Users\gkrul\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\engine\model.py", line 182, in __call__
    return self.predict(source, stream, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gkrul\AppData\Local\Programs\Pyth